In [31]:
import pandas as pd
import openpyxl as pxl
from json import loads, dumps

### Предобработка

In [4]:
df = pd.read_excel("https://rosstat.gov.ru/storage/mediabank/urov_10subg-nm.xlsx")

In [5]:
# Убираем строки, в которых есть данные по округам
df = df[~df.iloc[:, 0].str.contains('округ', na=False)]

In [6]:
# Удаление первых 2 и последних 2 строк
df = df.iloc[2:-2]

In [7]:
df.iloc[0,0] = 'Год'
df.iloc[1,0] = 'Квартал'

In [8]:
# Транспонирование таблицы и заполнение значений вниз
df_trans = df.transpose().fillna(method='ffill')

In [28]:
df_trans

,2,3,4,6,7,8,9,10,11,12,...,93,94,95,96,97,98,99,100,101,102
"Среднедушевые денежные доходы населения по субъектам \nРоссийской федерации, руб./месяц *",Год,Квартал,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,...,Республика Бурятия,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская авт.область
Unnamed: 1,2013 год,I квартал,21800,19980,16799,15802,17378,15015,20534,14954,...,Республика Бурятия,24322,Забайкальский край,28091,20868,25164,20707,38845,34900,17314
Unnamed: 2,2013 год,II квартал,24990,23494,18762,17319,20358,17333,21330,16723,...,Республика Бурятия,30924,Забайкальский край,36858,24512,28302,21476,42256,37268,20735
Unnamed: 3,2013 год,III квартал,25529,23113,18538,18270,21631,17493,21323,16956,...,Республика Бурятия,28973,Забайкальский край,34606,22491,27800,22187,41321,37331,19411
Unnamed: 4,2013 год,IV квартал,30533,26494,20401,20626,27339,20978,25738,19110,...,Республика Бурятия,38112,Забайкальский край,42473,26953,35492,27455,45358,41319,23648
Unnamed: 5,2013 год,2013 год,25684.374666,23246.766439,18707.421437,18057.844532,21682.906642,17754.109842,22242.843829,16968.160526,...,Республика Бурятия,30595.42978,Забайкальский край,35545.068503,23759.397989,29213.128366,23035.765409,42228.418189,37791.470002,20412.731357
Unnamed: 6,2014 год,I квартал,22457,20308,17256,16264,19743,16699,20816,16555,...,Республика Бурятия,25445,Забайкальский край,29558,25903,25768,19299,41193,35289,19308
Unnamed: 7,2014 год,II квартал,27059,24038,20949,19713,23367,20138,23091,18513,...,Республика Бурятия,32474,Забайкальский край,37508,27829,30753,25524,43894,43024,21331
Unnamed: 8,2014 год,III квартал,27965,25847,21816,19484,26213,19219,24069,19104,...,Республика Бурятия,30895,Забайкальский край,37896,26649,31401,27782,43890,45592,21254
Unnamed: 9,2014 год,IV квартал,32285,28931,22037,22445,30617,23035,29582,22159,...,Республика Бурятия,43546,Забайкальский край,50480,29387,36792,28848,49663,43434,24946


### Обработка столбца, содержащего кварталы

In [9]:
# Фильтрация строк
df = df_trans[~(df_trans[3].str.endswith('год') | df_trans[3].isnull())]

In [10]:
# Переименовываем индексы (заголовки столбцов)
df.columns = df.iloc[0]
df.drop(df.index[0])
df = df.iloc[1:]

In [11]:
# Замена значений в столбце с кварталами
df['Квартал'] = df['Квартал'].str.replace('IV', '4')
df['Квартал'] = df['Квартал'].str.replace('III', '3')
df['Квартал'] = df['Квартал'].str.replace('II', '2')
df['Квартал'] = df['Квартал'].str.replace('I', '1')

In [12]:
# Убираем слово "квартал" 
df['Квартал'] = df['Квартал'].str.replace('квартал', '', case=True)

In [13]:
# Переводим столбец "Квартал" в числовой формат
df['Квартал'] = pd.to_numeric(df['Квартал'], errors='coerce')

### Обрабатываем столбец с годом

In [14]:
# Замена незначащих знаков (звёздочки)
df['Год'] = df['Год'].str.replace('*', "")

C:\Users\ValeriiYS\AppData\Local\Temp\ipykernel_12268\1245002194.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Год'] = df['Год'].str.replace('*', "")


In [15]:
df['Год'] = df['Год'].str.replace('год', '', case=True)

In [16]:
df['Год'] = pd.to_numeric(df['Год'], errors='coerce')

In [17]:
# Фильтрация строк по условию
df = df[(df['Год'].isnull()) | (df['Год'] > 2018)]

In [18]:
df['Год'].fillna(0)

Unnamed: 31    2019
Unnamed: 32    2019
Unnamed: 33    2019
Unnamed: 34    2019
Unnamed: 36    2020
Unnamed: 37    2020
Unnamed: 38    2020
Unnamed: 39    2020
Unnamed: 41    2021
Unnamed: 42    2021
Unnamed: 43    2021
Unnamed: 44    2021
Unnamed: 46    2022
Unnamed: 47    2022
Unnamed: 48    2022
Unnamed: 49    2022
Unnamed: 51    2023
Unnamed: 52    2023
Unnamed: 53    2023
Unnamed: 54    2023
Name: Год, dtype: int64

In [19]:
df_melt = df.melt(id_vars=['Год','Квартал'], var_name='Region', value_name='Value')

In [20]:
# Замена двойных пробелов одинарными:
df_melt['Region'] = df_melt['Region'].replace('  ', ' ')

In [25]:
df_melt

,Год,Квартал,Region,Value
0,2019,1,Российская Федерация,30292
1,2019,2,Российская Федерация,34673
2,2019,3,Российская Федерация,35262
3,2019,4,Российская Федерация,41780
4,2020,1,Российская Федерация,32072
...,...,...,...,...
1675,2022,4,Еврейская авт.область,42388
1676,2023,1,Еврейская авт.область,35140
1677,2023,2,Еврейская авт.область,37480
1678,2023,3,Еврейская авт.область,39147


In [110]:
df_melt.to_excel(r'region_income.xlsx', index=False)

In [1]:
#df_melt.to_excel(r'\\dengi-srv\report\Выгрузки\ПДН\region_income.xlsx', index=False)

In [36]:
result = df_melt.to_json(orient='columns')
parsed = loads(result)
dumps(parsed, indent=2)  

'{\n  "\\u0413\\u043e\\u0434": {\n    "0": 2019,\n    "1": 2019,\n    "2": 2019,\n    "3": 2019,\n    "4": 2020,\n    "5": 2020,\n    "6": 2020,\n    "7": 2020,\n    "8": 2021,\n    "9": 2021,\n    "10": 2021,\n    "11": 2021,\n    "12": 2022,\n    "13": 2022,\n    "14": 2022,\n    "15": 2022,\n    "16": 2023,\n    "17": 2023,\n    "18": 2023,\n    "19": 2023,\n    "20": 2019,\n    "21": 2019,\n    "22": 2019,\n    "23": 2019,\n    "24": 2020,\n    "25": 2020,\n    "26": 2020,\n    "27": 2020,\n    "28": 2021,\n    "29": 2021,\n    "30": 2021,\n    "31": 2021,\n    "32": 2022,\n    "33": 2022,\n    "34": 2022,\n    "35": 2022,\n    "36": 2023,\n    "37": 2023,\n    "38": 2023,\n    "39": 2023,\n    "40": 2019,\n    "41": 2019,\n    "42": 2019,\n    "43": 2019,\n    "44": 2020,\n    "45": 2020,\n    "46": 2020,\n    "47": 2020,\n    "48": 2021,\n    "49": 2021,\n    "50": 2021,\n    "51": 2021,\n    "52": 2022,\n    "53": 2022,\n    "54": 2022,\n    "55": 2022,\n    "56": 2023,\n    "57